In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
from shapely.wkt import loads


In [ ]:
dbi = gpd.read_file('./data/Building Permits.geojson')

In [ ]:
permits = pd.read_csv('./data/SF_Planning_Permitting_Data.csv', low_memory=False)
permits['OPEN_DATE'] = pd.to_datetime(permits['OPEN_DATE'], errors='coerce')
permits['CLOSE_DATE'] = pd.to_datetime(permits['CLOSE_DATE'], errors='coerce')

In [ ]:
parcels = pd.read_csv('./data/Blue Sky Code and Inputs/SF_Logistic_Data.csv')

In [ ]:
allParcels = gpd.read_file('./data/Parcels   Active and Retired/parcels.shp')

In [ ]:
sites = gpd.read_file('./data/site_inventory/xn--Bay_Area_Housing_Opportunity_Sites_Inventory__20072023_-it38a.shp')

In [ ]:
tax = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2007-2008.xlsx')

In [ ]:
tax8 = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2008-2009.xlsx')

In [ ]:
use_codes = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2007-2008.xlsx', 
                          sheet_name='Class & Use Code')

In [ ]:
neighborhood_codes = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2007-2008.xlsx', 
                                   sheet_name='Neigborhood Code')

### Training Set is RHNA 4

In [ ]:
trainParcels = parcels[np.logical_and(parcels.year >= 2007, parcels.year <= 2015)]

In [ ]:
trainY = trainParcels.groupby('MapBlkLot_Master')['Developed'].agg(lambda x: x.ne(0).sum())

In [ ]:
trainY.sum()

In [ ]:
round(trainY.mean(), 3) * 100

In [ ]:
trainX = trainParcels[trainParcels.year == 2007]

No duplicative index.

In [ ]:
nunique_lots = trainParcels[trainParcels.year == 2007].MapBlkLot_Master.nunique()
n_lots = trainParcels[trainParcels.year == 2007].shape[0]
assert nunique_lots == n_lots

In [ ]:
trainX.MapBlkLot_Master.isin(trainY.index).mean()

In [ ]:
trainDf = pd.merge(trainX.drop('Developed', axis=1), trainY, left_on='MapBlkLot_Master', right_index=True)

In [ ]:
trainDf.Developed.value_counts()

In [ ]:
def clean_apn(apn):
    apn = ''.join(apn.split(' '))
    if len(apn) < 9:
        return apn
    block_length = 4
    #if apn[block_length].isalpha():
    #    return apn[:block_length] + apn[block_length+1:]
    return apn
 
    
tax['MapBlkLot_Master'] = tax.RP1PRCLID.apply(clean_apn)

Make BlueSky geospatial.

In [ ]:
trainDf.MapBlkLot_Master.isin(allParcels.blklot).mean()

In [ ]:
allParcels[allParcels.mapblklot == '0306026']

In [ ]:
date_cols = [c for c in allParcels.columns if 'date' in c]

In [ ]:
for date in date_cols:
    allParcels[date] = pd.to_datetime(allParcels[date], errors='coerce')

### Remove parcels deleted before 2007 and added after 2015

In [ ]:
allParcels.shape

In [ ]:
allParcels = allParcels[~(allParcels.date_map_d.dt.year <= 2007)]

In [ ]:
allParcels.shape

In [ ]:
allParcels = allParcels[~(allParcels.date_rec_d.dt.year <= 2007)]

In [ ]:
allParcels.shape

In [ ]:
allParcels = allParcels[~(allParcels.date_rec_a.dt.year >= 2015)]

In [ ]:
allParcels.shape

In [ ]:
allParcels = allParcels[~(allParcels.date_map_a.dt.year >= 2015)]

In [ ]:
allParcels.shape

In [ ]:
round((231457 - 215526)/231457, 2)

In [ ]:
round(allParcels.mapblklot.duplicated().mean(), 2)

In [ ]:
geoDf = trainDf.merge(allParcels, left_on='MapBlkLot_Master', right_on='mapblklot')

In [ ]:
geoDf.MapBlkLot_Master.duplicated().mean()

In [ ]:
geoDf.MapBlkLot_Master.value_counts()

In [ ]:
geoDf[geoDf.MapBlkLot_Master == '0306026'].date_map_a.value_counts()

In [ ]:
geoDf = geoDf.sort_values('date_map_a').groupby('MapBlkLot_Master').nth(0)

In [ ]:
geoDf.active.value_counts()

In [ ]:
allParcels.columns

In [ ]:
(geoDf.blklot == geoDf.mapblklot).mean()

Drop everything from AllParcels except mapblklot, blklot, and geometry.

In [ ]:
geoDf = geoDf.drop(allParcels.columns[2:-2], axis=1)

In [ ]:
trainDf = geoDf

### Developed parcels

In [ ]:
built = trainDf.loc[trainDf.Developed > 0,]

In [ ]:
built.MapBlkLot_Master.isin(tax.MapBlkLot_Master).mean().round(2)

In [ ]:
built.MapBlkLot_Master.str.len().value_counts()

In [ ]:
built.MapBlkLot_Master

In [ ]:
dbi.columns

In [ ]:
dbi['units'] = dbi.proposed_units.fillna(0).astype(float) - dbi.existing_units.fillna(0).astype(float)

In [ ]:
dbi.status.value_counts()

In [ ]:
dbi.columns

In [ ]:
date_cols = [c for c in dbi.columns if 'Date' in c]
dbi[date_cols] = dbi[date_cols].apply(pd.to_datetime)
dbi['apn'] = dbi['block'].astype(str) + dbi['lot'].astype(str)

In [ ]:
dbi['na_existing_units'] = dbi['existing_units'].isna()
relevant_uses = [
    'apartments', '1 family dwelling', '2 family dwelling',
    'residential hotel', 'misc group residns.', 'artist live/work',
    'convalescent home', 'accessory cottage', 'nursing home non amb',
    'orphanage', 'r-3(dwg) nursing', 'nursing home gt 6'
]
rhna_permits = dbi[
    (dbi['units'] > 0)
    & dbi['proposed_use'].isin(relevant_uses)
    & dbi['permit_type'].isin([1, 2, 3, 8])
].copy()

rhna_permits.query('not (`permit_type` == 8 and na_existing_units)', inplace=True)
rhna_permits.query('not (`permit_type` == 3 and na_existing_units)', inplace=True)


# Add / rename columns to fit ABAG format
rhna_permits['permyear'] = rhna_permits['issued_date'].dt.year

# Address is split up into multiple columns. Must re-combine.
id_on_street = rhna_permits['street_number'].astype(str) + " " + rhna_permits['street_number_suffix'].fillna("")
street = rhna_permits['street_name'] + ' ' + rhna_permits['street_suffix']
rhna_permits['address'] = id_on_street + ' ' + street

rhna_permits = rhna_permits.rename(columns={'Location': 'geometry'})

In [ ]:
rhna_permits = rhna_permits.loc[np.logical_and(rhna_permits.status_date.dt.year >= 2007,
                                               rhna_permits.status_date.dt.year <= 2015),]

In [ ]:
rhna_permits.apn

In [ ]:
rhna_permits.apn.isin(parcels.MapBlkLot_Master).mean()

In [ ]:
rhna_permits.units

In [ ]:
rhna_permits.apn.isin(built.MapBlkLot_Master).mean()

In [ ]:
built.MapBlkLot_Master.isin(rhna_permits.apn).mean()

In [ ]:
sum(rhna_permits.apn.duplicated())

In [ ]:
completed_projects = rhna_permits[rhna_permits['status'] == 'complete']

In [ ]:
dbi_units = completed_projects.groupby(['apn'], sort=False)['units'].median()

In [ ]:
built_poisson = built.merge(dbi_units, how='left', left_on='MapBlkLot_Master', right_on='apn')

In [ ]:
plt.hist(built_poisson.units)

In [ ]:
built_poisson.units.isna().mean().round(2)

In [ ]:
built_poisson

I still have 72% of the developed parcels to identify.

### Try using geometry

In [ ]:
dbi_units2 = completed_projects.groupby(['geometry'], sort=False)['units'].median()

In [ ]:
completed_projects.columns

In [ ]:
type(parcels)

#### How many matches do I get if I use SF Planning Permits

In the SF Planning Permits dataset, almost all mapblock lots are block + lot. Fewer are lot + block. And 29 have some non digit character I need to strip out.

Also, 7% blocklots are nans.

In [ ]:
permits[
    ~(np.logical_or(np.logical_or(permits.MAPBLOCKLOT == (permits.BLOCK + permits.LOT), 
                        permits.BLOCK.isna()), 
                   permits.MAPBLOCKLOT == (permits.LOT + permits.BLOCK)))
].shape

In [ ]:
ppermits = permits

In [ ]:
permits.MAPBLOCKLOT.isna().mean()

In [ ]:
permits.iloc[500:505, :]

In [ ]:
permits.RECORD_STATUS.value_counts()[:15]

In [ ]:
permits.RELATED_BUILDING_PERMIT

In [ ]:
permits.NEW_CONSTRUCTION.value_counts()

In [ ]:
permits.iloc[10:15, 20:]

In [ ]:
permits[permits.NUMBER_OF_UNITS > 0].NUMBER_OF_UNITS.sum()

In [ ]:
ppermits['NA_NUMBER_OF_UNITS_EXIST'] = ppermits['NUMBER_OF_UNITS_EXIST'].isna()
ppermits['units'] = ppermits['NUMBER_OF_UNITS'].fillna(0) - ppermits['NUMBER_OF_UNITS_EXIST'].fillna(0)

In [ ]:
ppermits.RESIDENTIAL_EXIST.value_counts()

In [ ]:
[c for c in ppermits.columns if 'ADD' in c]

In [ ]:
ppermits['NEW_CONSTRUCTION'] == 'CHECKED'

In [ ]:
statuses = ['Closed - Approved', 'Closed']
# Add / rename columns to fit ABAG format
ppermits['permyear'] =  ppermits['CLOSE_DATE'].dt.year
rhna_ppermits = ppermits[
    (ppermits['units'] > 0)
    & (ppermits['RECORD_STATUS'].isin(statuses)) 
    & (ppermits['permyear'] >= 2006)
].copy()



In [ ]:
rhna_ppermits.CLOSE_DATE

In [ ]:
built_poisson.MapBlkLot_Master.isin(permits.MAPBLOCKLOT).mean()

In [ ]:
built_poisson[
    built_poisson.MapBlkLot_Master.isin(rhna_ppermits.MAPBLOCKLOT) 
    & built_poisson.units.isna()
].shape[0]

In [ ]:
built_poisson.units.isna().sum()

In [ ]:
built_poisson.merge()